# Back-propagation, Automatic Differentiation and Gradient Descent
경사하강법으로 수치상의 최적점을 찾는 과정은 역전파 과정이었다. 즉, 예측 결과로부터 얻어지는 오차를 이용해 계산과정을 거슬러 올라가며 파라미터를 조정하는 **역**전파 방식이었다. 이를 차분식으로 갱신하는 방식으로 구현해보면, 시간이 많이 걸린다는 것을 알 수 있다. 

대신에, 훨씬 시간이 절약되고 효율적인 계산 알고리즘인 자동미분을 사용하여 Gradient Descent의 파라미터 조정값을 구할 수 있다. 자동미분의 원리를 이해하기 위해 예측결과를 내놓기 까지 순차적으로 계산을 하는 과정을 나타낸 Computational Graph를 이해하는 것이 중요하다.

그리고 나서 자동미분을 이용한 Gradient Descent 알고리즘을 구현해본다.

## 1. Computational Graph 계산 그래프
계산그래프는 계산 과정을 연산을 나타내는 노드node와 수행과정을 나타내는 방향성있는 엣지edge로 구성된 집합이다.

노드는 연산자(+, *, **,..)들 혹은 함수/합성함수(relu, sigmoid, softmax with loss, ...)를 나타낸다.

엣지는 노드끼리 연결하며 계산이 진행되는 방향을 묘사한다. 

입력갑/결과값은 노드와 묶어서 노드의 레이블로 표시하기도 하는가 하면, 엣지의 레이블로 표시하기도 하는데, 큰 차이는 없다.

계산그래프로 데이타의 변형과정을 한 눈에 구분할 수 있어서 게산 과정 알고리즘을 분해-이해하는데 유용하다.


### 1.1 순방향의 계산 그래프
순차적인 계산 결과를 진행하며, 중요한 노드를 중심으로 결과값들을 저장한다. 다음은 4 개의 노드로 구분하여 저장한 예시이다.
```python
a1 = np.dot(x.W1) + b1
z1 = sigmoid(a1)
a2 = np.dot(z1.W2) + b2
y = softmax(a2)
``` 
`np.dot`, `sigmoid`, `np.dot`, `sigmoid` 노드를 차례로 거쳐가도록 구현되었다.

`np.dot`은 3 개의 입력을 받고 1 개의 출력을 내므로 3 개의 in-edge 와 1 개의 out-edge가 연결되어 있다.

`sigmoid`, `softmax`는 in-edge와 out-edge 1개씩을 가진다.

### 1.2 역방향의 계산 그래프
합성함수의 미분과정이 chain rule을 따르는데, 이 과정이 역방향이다. 그래서 역방향의 계산그래프가 필요하고 이것으로 그래디언트를 구할 수 있게 되는 것이다.

$$
\frac{d}{dt}\Big( f\circ g \Big) = \frac{d}{dt}\Big(f(g(t))\Big) = \frac{df}{du}\frac{du}{dt}, \quad u = g(t)
$$

- 게다가 각 노드마다 이전 노드에서 구한 값($\frac{df}{du}$)을 현재노드에서 구한 값($\frac{du}{dt}$)에 곱해주기만 하면 되므로 얼마나 간단한가.
- 다변수 함수인 경우, chain rule 은 다음과 같다.
$$
\frac{\partial f}{\partial t} = \sum_i\frac{\partial f}{\partial u_i} \frac{\partial u_i}{\partial t}
$$ 
#### 1.2.1 Affine transformation의 계산 그래프
행렬변환의 다음과 같은 형태를 아파인 변환이라 하고 이에 대한 계산그래프를 구해보자.
$$\begin{align*}
\mathbf{X}\cdot\mathbf{W}\quad +\quad & \mathbf{b}  &= &\quad \mathbf{Y} \\
(n, d) (d, p)\quad& (p, )&\rightarrow & \quad(n, p)
\end{align*}$$
- 위 식의 경우 $b$ 항에 관해 broadcasting을 하게 되는데, 그 과정도 식으로 나타내보면, 
$$
\mathbf{1}\cdot \mathbf{b} = \mathbf{B}, \quad \text{ , } \mathbf{1} \equiv  \begin{bmatrix}
1 \\
\vdots \\
1
\end{bmatrix}_{n\times 1}
$$
으로 브로드캐스팅이 되므로, 다시 쓰면,

$$\begin{align*}
\mathbf{X}\cdot\mathbf{W}\quad +\quad & \mathbf{B}  &= &\quad \mathbf{Y} \\
(n, d) (d, p)\quad& (n,p)&\rightarrow & \quad(n, p)
\end{align*}$$
이렇게 나타낼 수 있다. 다음은 계산관련 notation을 간략히 설명한다.
>- Index Notation of a 2D-Array: 행렬모양의 2d-Array를 간단히 나타내기 위해 배열의 일반원소를 인덱스 2개를 사용하여 나타낸다.
>- Einstein summation convention을 사용하기 위해 행은 upper, 열은 lower 인덱스를 사용하므로 지수표현이 아닐 뿐 아니라
>- upper / lower 인덱스가 같은 문자일 경우 그 문자를 따라 합산하는 것임을 의미한다. 즉, 
>$$
>\mathbf{M} \equiv m^{i}_{j}, \quad \mathbf{X}\cdot\mathbf{W} = \sum_k x_{ik}w_{kj} \equiv x^{i}_{k}w^{k}_{j} = y^i_j = \mathbf{Y}
>$$
>$$
>\begin{align*}
>\mathbf{X}\cdot\mathbf{W} + \mathbf{b} = \mathbf{Y} \\ 
>d\mathbf{Y} = d\mathbf{X}\cdot\mathbf{W} + \mathbf{X}\cdot d\mathbf{W} + d\mathbf{B} \tag{1}  \\
>\frac{\partial L}{\partial \mathbf{Y}} = \frac{\partial L}{\partial y^i_j} \equiv \Big[ \frac{\partial L}{\partial y_{ij}}\Big]
>\end{align*}
>$$
- Affine transformation의 gradient를 구하는 $\cdot$ `dot` 의 계산 그래프를 살펴보자. 아파인 노드로 손실 변화분인 $\frac{\partial L}{\partial \mathbf{Y}}$ 가 유입되는 경우, $\mathbf{X}$, $\mathbf{W}$, $\mathbf{b}$ 로 분기되는 값은 무엇일까?
$$\begin{align*}
\frac{\partial L}{\partial \mathbf{X}} &\equiv \frac{\partial L}{\partial x^i_j} = \frac{\partial L}{\partial y^p_q}\frac{\partial y^p_q}{\partial x^i_j} = \frac{\partial L}{\partial y^p_q}\frac{\partial (x^p_k w^k_q)}{\partial x^i_j} = \frac{\partial L}{\partial y^p_q}\delta^p_i\delta^j_k w^k_q = \frac{\partial L}{\partial y^i_q}w^j_q = \frac{\partial L}{\partial \mathbf{Y}}\cdot \mathbf{W}^t \\
\frac{\partial L}{\partial \mathbf{W}} &\equiv \frac{\partial L}{\partial w^i_j} = \frac{\partial L}{\partial y^p_q}\frac{\partial y^p_q}{\partial w^i_j} = \frac{\partial L}{\partial y^p_q}\frac{\partial (x^p_k w^k_q)}{\partial w^i_j} = \frac{\partial L}{\partial y^p_q}\delta^k_i\delta^j_q x^p_k = \frac{\partial L}{\partial y^p_j}x^p_i = \mathbf{X}^t \cdot\frac{\partial L}{\partial \mathbf{Y}} \tag{2}\\
\frac{\partial L}{\partial \mathbf{B}} &\equiv \frac{\partial L}{\partial b^i_j} = \frac{\partial L}{\partial y^p_q} \frac{\partial y^p_q}{\partial b^i_j} = \frac{\partial L}{\partial y^p_q}\frac{\partial b^p_q}{\partial b^i_j}= \frac{\partial L}{\partial y^p_q}\delta^p_i\delta^q_j = \frac{\partial L}{\partial \mathbf{Y}} \cdot \mathbf{I} 
\end{align*}$$
한편, 
$$
\frac{\partial B}{\partial b} \equiv \frac{\partial B^i_j}{\partial b_k}= \frac{\partial }{\partial b_k}\Big( \mathbf{1}^i b_j\Big) = \mathbf{1}^i \delta^k_j
$$ 
이므로 마지막 식을 대입하면, 
$$
\frac{\partial L}{\partial \mathbf{b}} \equiv \frac{\partial L}{\partial b_k} = \Big(\frac{\partial L}{\partial \mathbf{Y}}\frac{\partial \mathbf{Y}}{\partial \mathbf{B}}\Big) \frac{\partial \mathbf{B}}{\partial \mathbf{b}} = \Big(\frac{\partial L}{\partial y^i_j} \Big) \mathbf{1}^i \delta^k_j = \frac{\partial L}{\partial y^i_k}  \mathbf{1}^i = \mathbf{1}^t \cdot \frac{\partial L}{\partial \mathbf{Y}} = \sum\limits_{i=0}^{n-1} \frac{\partial L}{\partial y^i_k}
$$
- $x_0 = 1$, $w_{0k}=b_k$ 로 하여 각각 $\mathbf{X}$,$\mathbf{W}$ 에 최초 열과 행으로 추가한 $\mathbf{X}^{\prime}$, $\mathbf{W}^{\prime}$ 으로 $\mathbf{X}^{\prime}\cdot\mathbf{W}^{\prime}=\mathbf{Y}$ 와 같이 구성한 식으로도 같은 gradient 결과가 나옴을 확인할 수 있다. 
- 결과를 보면, 행렬의 덧셈 노드는 실수의 덧셈 노드처럼 흘러온 값을 그대로 분기시키는 것을 알 수 있다.
- 마찬가지로 $\cdot$ `dot` 노드는 실수의 곲셈 노드처럼 흘러온 값에 상대방 값을 `dot` 연산을 한다는 것을 알 수 있다.  
- 따라서 Affine 노드의 경우 다음과 같이 gradient를 구하는 코드를 작성할 수 있을 것이다.

```python
grads['W2'] = np.dot(z1.T, dy)
grads['b2'] = np.sum(dy, axis=0)
```

#### 1.2.2 Loss $\circ$ softmax 의 계산그래프
이전에 살펴보았듯이 Loss 함수로 cross-entropy-error를 사용하는데 이는 수식의 특성상 softmax함수와 긴밀하게 연관이 있기 때문이다. 이 점은 Loss, CEE의 gradient를 따로 구하여 곱하는 것보다 한 덩어리로서 gradient를 구하는 것이 더 효율적이란 사실로 이어진다.

$$
a \overset{Softmax}\longrightarrow y \overset{CEE}\longrightarrow L
$$

$$\begin{align*}
\frac{\partial L}{\partial a_k} = \frac{\partial L}{\partial y_p}\frac{\partial y_p}{\partial a_k} = \frac{\partial L}{\partial y_k}\frac{\partial y_k}{\partial a_k} + \frac{\partial L}{\partial y_{\tilde{k}}}\frac{\partial y_{\tilde{k}}}{\partial a_k} = - \frac{t_k}{y_k}y_k(1-y_k) - \frac{t_{\tilde{k}}}{y_{\tilde{k}}}\Big(-y_k y_{\tilde k}\Big) = -t_k(1-y_k) + t_{\tilde{k}}y_k = y_k - t_k
\end{align*}$$
- 여기서, 인덱스 $\tilde{k}$ 는 $k$ 이외의 인덱스들을 일반적으로 표현한 것이고, 그래서 당연하게도 $t_k + t_{\tilde{k}} = 1$ 이 된다. One-hot 인코딩은 하나만 1이고 나머지는 0임을 상기하자.
- 이렇게 간단하게 도출이 되므로 구현할 때 함께 구현한다.
```python
    batch_size = self.t.shape[0]
    dx = (self.y - self.t) / batch_size
```
- `t`가 one-hot 인코딩이 아니고 정답 숫자 인코딩일 경우, one-hot으로 변경해주고 구한다. 이 때, t의 값은 정답인덱스에서만 1이고 나머지에서는 0 이므로 그래디언트 값인 $y_k - t_k$는 `y`의 정답인텍스에서만  `value - 1` 으로 변하기 때문에 다음처럼 작성할 수 있다. 
```python
da = self.y.copy()
da[np.arange(batch_size), self.t] -= 1 
da = da / batch_size
```

### 2. Autodiff을 사용한 Gradient를 가지는 클래스로 기능별 layer 구현하기
지금까지 요약해 보면, 계산 단위별로 local하게 numerical gradient를 구할 수가 있고,  필요한 만큼 쌓아올려 원하는 깊이의 gradient를 구할 수 있게 되었다. 이렇게 할 수 있었던 것은 Chain rule이 보장해주기 때문이었다.

이렇게 분할정복이 가능하기 때문에 주요 계산단위별로 클래스를 구성하여 각 클래스에 Gradient 메서드 즉, backward propagating autodifferentiation 메서드를 구현해 넣으면, 원하는 만큼 조립하여 network의 게층 구조를 만들 수 있다.

In [1]:
import numpy as np
import matplotlib.pyplot as plt 
from collections import OrderedDict

In [2]:
from common.functions import softmax, cross_entropy_error


class Multiplication:   # swap multiplier
    def __init__(self) -> None:
        self.x = None
        self.y = None
    def forward(self, x, y):
        self.x = x
        self.y = y
        return x * y
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        return dx, dy

class Maximum:          # gradient router
    def __init__(self) -> None:
        self.x = None
        self.y = None
        self.maskx = None
        self.masky = None
    def forward(self, x, y):
        self.x = x
        self.y = y
        self.maskx = np.array(x >= y, dtype=np.int16)
        self.masky = np.array(x <= y, dtype=np.int16)
        return np.maximum(x, y)
    def backward(self, dout):
        dx = dout * self.maskx
        dy = dout * self.masky
        return dx, dy

class Addition:     # gradient distributor
    def __init__(self) -> None:
        self.x = None
        self.y = None
    def forward(self, x, y):
        self.x = x
        self.y = y
        return x + y
    def backward(self, dout):
        dx = dout
        dy = dout
        return dx, dy

class Copy:         # gradient adder
    def __init__(self) -> None:
        self.x = None
        self.dout1 = None
        self.dout2 = None 
    def forward(self, x):
        self.x = x
        self.dout1 = x.copy()
        self.dout2 = x.copy()
        return x, x
    def backward(self, dout1, dout2):
        return dout1 + dout2

class Step:        # 
    def __init__(self) -> None:
        self.x = None
    def forward(self, x):
        x = np.array(x)
        self.x = x
        return np.array(x > 0, dtype=np.int32)
    def backward(self, dout):
        return self.x * 0

class Relu:
    def __init__(self) -> None:
        self.mask = None
    def forward(self, x):
        self.mask = np.array(x > 0, dtype=np.int32)
        return np.maximum(x, 0)
    def backward(self, dout):
        return dout * self.mask

class Affine:
    def __init__(self, W, b) -> None:
        self.W = W   # (d, p)
        self.b = b   # (p, )
        self.X = None   # (n, d)
        self.dW = None
        self.dX = None
        self.db = None
    def forward(self, X):
        self.X = X   # (n, d) or (d, )
        out = np.dot(self.X, self.W) + self.b # (n, p) + (p, ) -> (n, p) broadcasting
        return out
    def backward(self, dout):
        self.dX = np.dot(dout,self.W.T)
        # ndim == 1인 경우 문제가 생긴다. 이를 해결하기 위한 조건문.
        if dout.ndim == 2: # 즉, batch로 입력되는 경우이다.
            self.dW = np.dot(self.X.T, dout)
            self.db = np.sum(dout, axis=0)  # forward시 브로드캐스팅으로 copy node 가 생략되었었다고 보고 adder를 적용
        elif dout.ndim == 1:
            self.dW = np.dot(self.X.reshape(self.X.size, 1), dout.reshape(1, dout.size))
            self.db = dout         
        else:
            print("Type or dimension is wrong.")
            raise TypeError
        return self.dX #, self.dW, self.db

class Sigmoid:
    def __init__(self) -> None:
        self.out = None
    def forward(self, x):
        self.out = 1/(1+np.exp(-x))
        return self.out
    def backward(self, dout):
        return dout * self.out * (1.0 - self.out)

class CEE_Softmax:
    def __init__(self) -> None:
        self.y = None
        self.t = None
        self.loss = None
    def forward(self, a, t):
        self.t = t
        self.y = softmax(a)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    def backward(self, dout=1.0):
        batch_size = self.t.shape[0]
        if self.y.size == self.t.size:  # one-hot일 경우에 같다.
            da = (self.y - self.t)/batch_size
        else:
            da = self.y.copy()
            da[np.arange(batch_size), self.t] = 1 # y 사본으로 t one-hot 만들기
            da = da / batch_size
        return da
        

In [3]:
from collections import OrderedDict
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01) -> None:
        # initialize weights
        self.params = {}
        self.params['W1'] = weight_init_std*np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std*np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

        # layers
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.last_layer = CEE_Softmax()


    def predict(self, X):
        for layer in self.layers.values():
            X = layer.forward(X)  # take the last result of X
        return X   
        
    def loss(self, X, t): # ->> predict, CEE
        Y = self.predict(X)
        return self.last_layer.forward(Y, t)
        
    def accuracy(self, X, t): # ->> predict, loss
        batch_size = X.shape[0] if X.ndim != 1 else 1
        Y = self.predict(X) # shape: X (n, d) W (d p) Y (n, p)
        Y = np.argmax(Y, axis= -1)
        if t.ndim != Y.ndim: t = np.argmax(t, axis= -1)
        accuracy = np.sum(Y.astype(np.int64)==t.astype(np.int64)) / float(batch_size)
        return accuracy

    
    def gradient(self, X, t):  # ->> loss
        # forward
        self.loss(X, t)
        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

In [4]:
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size/batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # Gradient
    grad = network.gradient(x_batch, t_batch)
    
    # Renewal
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    # Loss and Accuracy
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print('\r', train_acc, test_acc, end='')

 0.9782666666666666 0.96843